In [1]:
import numpy as np
import graphinglib as gl
import dill
import pyregion

from src.statistics.advanced_stats import increments
from src.hdu.maps.map import Map

In [2]:
def get_plottables(data: np.ndarray, number_of_bins: int, increment: float) -> [gl.Histogram, gl.Text]:
    sorted_data = np.sort(np.concatenate((data, -data)))

    hist = gl.Histogram(
        sorted_data,
        number_of_bins=number_of_bins,
        show_params=False
    )
    hist.add_pdf(show_mean=False, show_std=False)
    text = gl.Text(
        hist.bin_edges[0]*0.8,
        np.max(hist.bin_heights)*0.8,
        rf"$\Delta={increment}$"
    )

    return [hist, text]

### All increments

In [3]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("global_region", None, 40),
    ("diffuse_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 30),
    ("central_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 20),
    ("filament_region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 10)
]

In [ ]:
with open("applications/sh158/saved_data/increments.gz", "wb") as f:
    dill.dump(increments(turbulence.data), f)

In [4]:
# with open("applications/sh158/saved_data/increments.gz", "rb") as f:
#     increments_data = dill.load(f)

all_figs = []

for name, region, number_of_bins in regions:
    increments_data = increments(turbulence.get_masked_region(region).data)

    figs = []

    for increment in [2,5,10,20]:
        fig = gl.Figure(x_label="Increment", y_label="PDF", title=name, log_scale_y=True)
        plottables = get_plottables(increments_data[increment], number_of_bins, increment)
        plottables[0].number_of_bins = number_of_bins
        fig.add_elements(*plottables)
        figs.append(fig)
        
    all_figs.extend(figs)

    multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
    # multifig.show()
    multifig.save(f"figures/sh158/advanced_stats/increments/{name}.pdf")

giga_multifig = gl.MultiFigure.from_grid(all_figs, (4,4), (20,11))
giga_multifig.save(f"figures/sh158/advanced_stats/increments/everything.pdf")

In [ ]:
increments(np.array([
    [0,1,4],
    [4,5,2],
    [4,8,1]
]))

gl.Vlines(5, )